In [1]:
from liquidhandling import SoloSoft, SoftLinx
from liquidhandling import *  # replace with plate types used 

#* Program Variables ------------------
stock_start_column = 1
spot_z_shift = 2.4  # no additional height added -> should touch agar?

media_aspirate_column = 1 
first_column_transfer_volume = 100
dilution_media_volume = 90
dilution_transfer_volume = 10
stock_mix_volume = 50
dilution_mix_volume = 30  # 50uL tips used
num_mixes = 5

default_z_shift = 2
spot_volume = 3.5
#* ---------------------------------------
soloSoft = SoloSoft( 
    filename="dilute_then_spot_STEP1.hso",
    plateList=[
        "TipBox.200uL.Corning-4864.orangebox",
        "Empty",
        "Reservoir.12col.Agilent-201256-100.BATSgroup",
        "Empty",
        "Plate.96.Corning-3635.ClearUVAssay",
        "Plate.96.Corning-3635.ClearUVAssay",
        "TipBox.50uL.Axygen-EV-50-R-S.tealbox",
        "AgarPlate.40mL.OmniTray-242811.ColonyPicker",
    ]
)

#* STEP 1: Dispense diluent into whole plate -> MUST CHECK/REFILL BETWEEN CREATING EACH PLATE -------------
    # Don't dispense diluent into first column of each half plate -> will be pure stock
soloSoft.getTip()  
for i in range(2,7):  # first half of the plate, columns 2-6  
    soloSoft.aspirate(
        position="Position3", 
        aspirate_volumes=Reservoir_12col_Agilent_201256_100_BATSgroup().setColumn(media_aspirate_column, dilution_media_volume), 
        aspirate_shift=[0,0,4], 
    )
    soloSoft.dispense(
        position="Position6", 
        dispense_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn(i, dilution_media_volume), 
        dispense_shift=[0,0,default_z_shift],
    )
media_aspirate_column += 1
for i in range(8,13):  # second half of the plate, columns 8-12
    soloSoft.aspirate(
        position="Position3", 
        aspirate_volumes=Reservoir_12col_Agilent_201256_100_BATSgroup().setColumn(media_aspirate_column, dilution_media_volume), 
        aspirate_shift=[0,0,4], 
    )
    soloSoft.dispense(
        position="Position6", 
        dispense_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn(i, dilution_media_volume), 
        dispense_shift=[0,0,default_z_shift],
    )
soloSoft.shuckTip()
soloSoft.savePipeline()

#* STEP 2: Serial Dilute into dilution plate and spot onto agar plate (COLUMNS 1-6) -------------------------
i = 1
soloSoft = SoloSoft( 
    filename="dilute_then_spot_STEP2.hso",
    plateList=[
        "TipBox.200uL.Corning-4864.orangebox",
        "Empty",
        "Reservoir.12col.Agilent-201256-100.BATSgroup",
        "Empty",
        "Plate.96.Corning-3635.ClearUVAssay",
        "Plate.96.Corning-3635.ClearUVAssay",
        "TipBox.50uL.Axygen-EV-50-R-S.tealbox",
        "AgarPlate.40mL.OmniTray-242811.ColonyPicker",
    ]
)

# set up first column of dilution plate -> pure stock, no dilution (100uL transfer volume)
soloSoft.getTip() # 200uL tips
soloSoft.aspirate(
    position="Position5", 
    aspirate_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn(stock_start_column, first_column_transfer_volume),
    aspirate_shift=[0,0,default_z_shift], 
    mix_at_start=True, 
    mix_volume=stock_mix_volume, 
    mix_cycles=num_mixes, 
    dispense_height=default_z_shift, 
)
soloSoft.dispense(
    position="Position6", 
    dispense_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn((6*(i-1))+1, first_column_transfer_volume),
    dispense_shift=[0,0,default_z_shift], 
    mix_at_finish=True,
    mix_volume=dilution_mix_volume, 
    mix_cycles=num_mixes,
    aspirate_height=default_z_shift, 
)
print("Prepare the first dilution column: ")
print("\t From clear UV column ( " + str(stock_start_column) + " ) to clear dilution UV column ( " + str((6*(i-1))+1) + " )")


print("Diluting and spotting (COLUMNS 1-6): ")
# LOOP: spot dilution onto agar plate and complete next dilution in serial dilution plate
for j in range(1,6): # 1,2,3,4,5
    soloSoft.getTip("Position7")

    # spot 3.5uL onto agar plate 
    soloSoft.aspirate(
        position="Position6", 
        aspirate_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn((6*(i-1))+j, spot_volume), 
        aspirate_shift=[0,0,default_z_shift], 
    )
    soloSoft.dispense(
        position="Position8",
        dispense_volumes=AgarPlate_40mL_OmniTray_242811_ColonyPicker().setColumn((6*(i-1))+j, spot_volume),
        dispense_shift=[0,0,spot_z_shift], 
    )
    
    #soloSoft.getTip("Position7") <----------- uncomment this for new tip between spot and dilute steps
    # use same tips to create the next dilution
    soloSoft.aspirate(
        position="Position6",
        aspirate_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn((6*(i-1))+j, dilution_transfer_volume), 
        aspirate_shift=[0,0,default_z_shift],
    )
    soloSoft.dispense(
        position="Position6",
        dispense_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn((6*(i-1))+j+1, dilution_transfer_volume), 
        dispense_shift=[0,0,default_z_shift], 
        mix_at_finish=True,
        mix_cycles=num_mixes,
        mix_volume=dilution_mix_volume, 
        aspirate_height=default_z_shift, 
    )

    print("\t Spot:  From clear UV column ( " + str((6*(i-1))+j) + " ) to agar plate ( " + str((6*(i-1))+j) + " )")
    print("\t Dilute:  From clear UV column ( " + str((6*(i-1))+j) + " ) to clear UV column ( " + str((6*(i-1))+j+1) + " )")
        
# spot the last column onto the agar plate
soloSoft.aspirate(
    position="Position6",
    aspirate_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn((6*(i-1))+6, spot_volume), 
    aspirate_shift=[0,0,default_z_shift], 
)
soloSoft.dispense(
    position="Position8", 
    dispense_volumes=AgarPlate_40mL_OmniTray_242811_ColonyPicker().setColumn((6*(i-1))+6, spot_volume),
    dispense_shift=[0,0,spot_z_shift], 
)

print("Spot from the last column (COLUMN 6)")
print("\t Spot:  From clear UV column ( " + str((6*(i-1))+6) + " ) to agar plate ( " + str((6*(i-1))+6) + " )")

# mimic loop like variable changes
i += 1 
stock_start_column += 1

soloSoft.shuckTip()
soloSoft.savePipeline()

#* STEP 3: Serial Dilute into dilution plate and spot onto agar plate (COLUMNS 7-12) -------------------------


soloSoft = SoloSoft( 
    filename="dilute_then_spot_STEP3.hso",
    plateList=[
        "TipBox.200uL.Corning-4864.orangebox",
        "Empty",
        "Reservoir.12col.Agilent-201256-100.BATSgroup",
        "Empty",
        "Plate.96.Corning-3635.ClearUVAssay",
        "Plate.96.Corning-3635.ClearUVAssay",
        "TipBox.50uL.Axygen-EV-50-R-S.tealbox",
        "AgarPlate.40mL.OmniTray-242811.ColonyPicker",
    ]
)


# set up first column of dilution plate -> pure stock, no dilution (100uL transfer volume)
soloSoft.getTip() # 200uL tips
soloSoft.aspirate(
    position="Position5", 
    aspirate_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn(stock_start_column, first_column_transfer_volume),
    aspirate_shift=[0,0,default_z_shift], 
    mix_at_start=True, 
    mix_volume=stock_mix_volume, 
    mix_cycles=num_mixes, 
    dispense_height=default_z_shift, 
)
soloSoft.dispense(
    position="Position6", 
    dispense_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn((6*(i-1))+1, first_column_transfer_volume),
    dispense_shift=[0,0,default_z_shift], 
    mix_at_finish=True,
    mix_volume=dilution_mix_volume, 
    mix_cycles=num_mixes,
    aspirate_height=default_z_shift, 
)

print("\nPrepare the first dilution column - second half")
print("\t From clear UV column ( " + str(stock_start_column) + " ) to clear dilution UV column ( " + str((6*(i-1))+1) + " )")
print("\nDiluting and spotting (COLUMNS 7-12): ")
    # LOOP: spot dilution onto agar plate and complete next dilution in serial dilution plate
for j in range(1,6): # 1,2,3,4,5
    soloSoft.getTip("Position7")

    # spot 3.5uL onto agar plate 
    soloSoft.aspirate(
        position="Position6", 
        aspirate_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn((6*(i-1))+j, spot_volume), 
        aspirate_shift=[0,0,default_z_shift], 
    )
    soloSoft.dispense(
        position="Position8",
        dispense_volumes=AgarPlate_40mL_OmniTray_242811_ColonyPicker().setColumn((6*(i-1))+j, spot_volume),
        dispense_shift=[0,0,spot_z_shift], 
    )
    
    #soloSoft.getTip("Position7") <----------- uncomment this for new tip between spot and dilute steps
    # use same tips to create the next dilution
    soloSoft.aspirate(
        position="Position6",
        aspirate_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn((6*(i-1))+j, dilution_transfer_volume), 
        aspirate_shift=[0,0,default_z_shift],
    )
    soloSoft.dispense(
        position="Position6",
        dispense_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn((6*(i-1))+j+1, dilution_transfer_volume), 
        dispense_shift=[0,0,default_z_shift], 
        mix_at_finish=True,
        mix_cycles=num_mixes,
        mix_volume=dilution_mix_volume, 
        aspirate_height=default_z_shift, 
    )

    print("\t Spot:  From clear UV column ( " + str((6*(i-1))+j) + " ) to agar plate ( " + str((6*(i-1))+j) + " )")
    print("\t Dilute:  From clear UV column ( " + str((6*(i-1))+j) + " ) to clear UV column ( " + str((6*(i-1))+j+1) + " )")
        
# spot the last column onto the agar plate
soloSoft.aspirate(
    position="Position6",
    aspirate_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn((6*(i-1))+6, spot_volume), 
    aspirate_shift=[0,0,default_z_shift], 
)
soloSoft.dispense(
    position="Position8", 
    dispense_volumes=AgarPlate_40mL_OmniTray_242811_ColonyPicker().setColumn((6*(i-1))+6, spot_volume),
    dispense_shift=[0,0,spot_z_shift], 
)

print("Spot from the last column (COLUMN 6)")
print("\t Spot:  From clear UV column ( " + str((6*(i-1))+6) + " ) to agar plate ( " + str((6*(i-1))+6) + " )")

soloSoft.shuckTip()
soloSoft.savePipeline()

# LOAD PROTOCOL STEPS 1-3 IN SOFTLINX
softLinx = SoftLinx("Dilute then Spot Steps 1-3", "dilute_then_spot.slvp")
softLinx.soloSoftRun("C:\\Users\\svcaibio\\Dev\\liquidhandling\\example\\other_protocols\\dilute_then_spot\\dilute_then_spot_STEP1.hso")
softLinx.soloSoftRun("C:\\Users\\svcaibio\\Dev\\liquidhandling\\example\\other_protocols\\dilute_then_spot\\dilute_then_spot_STEP2.hso")
softLinx.soloSoftRun("C:\\Users\\svcaibio\\Dev\\liquidhandling\\example\\other_protocols\\dilute_then_spot\\dilute_then_spot_STEP3.hso")
softLinx.saveProtocol()

Prepare the first dilution column: 
	 From clear UV column ( 1 ) to clear dilution UV column ( 1 )
Diluting and spotting (COLUMNS 1-6): 
	 Spot:  From clear UV column ( 1 ) to agar plate ( 1 )
	 Dilute:  From clear UV column ( 1 ) to clear UV column ( 2 )
	 Spot:  From clear UV column ( 2 ) to agar plate ( 2 )
	 Dilute:  From clear UV column ( 2 ) to clear UV column ( 3 )
	 Spot:  From clear UV column ( 3 ) to agar plate ( 3 )
	 Dilute:  From clear UV column ( 3 ) to clear UV column ( 4 )
	 Spot:  From clear UV column ( 4 ) to agar plate ( 4 )
	 Dilute:  From clear UV column ( 4 ) to clear UV column ( 5 )
	 Spot:  From clear UV column ( 5 ) to agar plate ( 5 )
	 Dilute:  From clear UV column ( 5 ) to clear UV column ( 6 )
Spot from the last column (COLUMN 6)
	 Spot:  From clear UV column ( 6 ) to agar plate ( 6 )

Prepare the first dilution column - second half
	 From clear UV column ( 2 ) to clear dilution UV column ( 7 )

Diluting and spotting (COLUMNS 7-12): 
	 Spot:  From clear UV 